# EXPLORATION_09

MF 모델 학습 방법을 토대로, 내가 좋아할 만한 영화 추천 시스템을 제작해 보겠습니다.

## 1) 데이터 준비와 전처리
* 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.
* 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해볼 수 있습니다.
* 별점을 시청횟수로 해석해서 생각하겠습니다.
* 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.


* Movielens 데이터는 rating.dat 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있습니다.



In [67]:
import pandas as pd
import numpy as np
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [68]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [69]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [70]:
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [71]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [72]:
# 검색을 쉽게하기 위해 영화제목 문자열을 소문자로 바꾸기
movies['title'] = movies['title'].str.lower()
movies

,movie_id,title,genre
0,1,toy story (1995),Animation|Children's|Comedy
1,2,jumanji (1995),Adventure|Children's|Fantasy
2,3,grumpier old men (1995),Comedy|Romance
3,4,waiting to exhale (1995),Comedy|Drama
4,5,father of the bride part ii (1995),Comedy
...,...,...,...
3878,3948,meet the parents (2000),Comedy
3879,3949,requiem for a dream (2000),Drama
3880,3950,tigerland (2000),Drama
3881,3951,two family house (2000),Drama


* 여기까지가 전처리입니다. 이후에는 이전 스텝에 소개했던 것과 동일한 방식으로 MF model을 구성하여 내가 좋아할 만한 영화를 추천해 볼 수 있습니다.

## 2) 분석해봅시다
* ratings에 있는 유니크한 영화 개수
* rating에 있는 유니크한 사용자 수
* 가장 인기 있는 영화 30개(인기순)


In [73]:
# ratings에 있는 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

In [74]:
# rating에 있는 유니크한 사용자 수
ratings['user_id'].nunique()

6039

In [75]:
ratings

,user_id,movie_id,count,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000203,6040,1090,3,956715518
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [76]:
movies

,movie_id,title,genre
0,1,toy story (1995),Animation|Children's|Comedy
1,2,jumanji (1995),Adventure|Children's|Fantasy
2,3,grumpier old men (1995),Comedy|Romance
3,4,waiting to exhale (1995),Comedy|Drama
4,5,father of the bride part ii (1995),Comedy
...,...,...,...
3878,3948,meet the parents (2000),Comedy
3879,3949,requiem for a dream (2000),Drama
3880,3950,tigerland (2000),Drama
3881,3951,two family house (2000),Drama


In [77]:
# ratings와 movies DataFrame 합치기
movie_rate = pd.merge(ratings,movies)
movie_rate

,user_id,movie_id,count,timestamp,title,genre
0,1,1193,5,978300760,one flew over the cuckoo's nest (1975),Drama
1,2,1193,5,978298413,one flew over the cuckoo's nest (1975),Drama
2,12,1193,4,978220179,one flew over the cuckoo's nest (1975),Drama
3,15,1193,4,978199279,one flew over the cuckoo's nest (1975),Drama
4,17,1193,5,978158471,one flew over the cuckoo's nest (1975),Drama
...,...,...,...,...,...,...
836473,5851,3607,5,957756608,one little indian (1973),Comedy|Drama|Western
836474,5854,3026,4,958346883,slaughterhouse (1987),Horror
836475,5854,690,3,957744257,"promise, the (versprechen, das) (1994)",Romance
836476,5938,2909,4,957273353,"five wives, three secretaries and me (1998)",Documentary


In [78]:
# 필요없는 컬럼 제거
movie_rate.drop(['movie_id','timestamp','genre'],axis=1,inplace=True)
movie_rate.head()

,user_id,count,title
0,1,5,one flew over the cuckoo's nest (1975)
1,2,5,one flew over the cuckoo's nest (1975)
2,12,4,one flew over the cuckoo's nest (1975)
3,15,4,one flew over the cuckoo's nest (1975)
4,17,5,one flew over the cuckoo's nest (1975)


In [79]:
# 가장 인기 있는 영화 30개(인기순)
movie_count =movie_rate.groupby('title')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

title
american beauty (1999)                                   3211
star wars: episode iv - a new hope (1977)                2910
star wars: episode v - the empire strikes back (1980)    2885
star wars: episode vi - return of the jedi (1983)        2716
saving private ryan (1998)                               2561
terminator 2: judgment day (1991)                        2509
silence of the lambs, the (1991)                         2498
raiders of the lost ark (1981)                           2473
back to the future (1985)                                2460
matrix, the (1999)                                       2434
jurassic park (1993)                                     2413
sixth sense, the (1999)                                  2385
fargo (1996)                                             2371
braveheart (1995)                                        2314
men in black (1997)                                      2297
schindler's list (1993)                                  2257
pr

In [80]:
# 유저별 몇 개의 영화를 시청했는지에 대한 통계
user_count = movie_rate.groupby('user_id')['title'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: title, dtype: float64

In [81]:
# 유저별 시청횟수(count) 중앙값에 대한 통계
user_median = movie_rate.groupby('user_id')['count'].median()
user_median.describe()

count    6039.000000
mean        4.055970
std         0.432143
min         3.000000
25%         4.000000
50%         4.000000
75%         4.000000
max         5.000000
Name: count, dtype: float64

## 3) 내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다

In [82]:
# 영화 검색
movies['title'].loc[movies['title'].str.contains('mummy') ]

2548            mummy, the (1999)
2564            mummy, the (1932)
2565            mummy, the (1959)
2566    mummy's curse, the (1944)
2567    mummy's ghost, the (1944)
2568     mummy's hand, the (1940)
2569     mummy's tomb, the (1942)
Name: title, dtype: object

In [83]:
# 내가 선호하는 영화
my_favorite = ['titanic (1997)' , 'beauty and the beast (1991)' ,'truman show, the (1998)' ,'aladdin (1992)' ,'mummy, the (1999)']

# 'jeena'라는 user_id가 위 영화를 5회씩 시청했다고 가정하겠습니다.
my_movielist = pd.DataFrame({'user_id': ['jeena']*5, 'title': my_favorite, 'count':[5]*5})

if not movie_rate.isin({'user_id':['jeena']})['user_id'].any():  # user_id에 'jeena'이라는 데이터가 없다면
    movie_rate = movie_rate.append(my_movielist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

movie_rate.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,count,title
836473,5851,5,one little indian (1973)
836474,5854,4,slaughterhouse (1987)
836475,5854,3,"promise, the (versprechen, das) (1994)"
836476,5938,4,"five wives, three secretaries and me (1998)"
836477,5948,5,identification of a woman (identificazione di ...
0,jeena,5,titanic (1997)
1,jeena,5,beauty and the beast (1991)
2,jeena,5,"truman show, the (1998)"
3,jeena,5,aladdin (1992)
4,jeena,5,"mummy, the (1999)"


## 모델에 활용하기 위한 전처리 (실습)
우리가 다루는 데이터에서는 user와 title 각각에 번호를 붙이고 싶습니다. 보통 이런 작업을 indexing이라고 합니다.

In [84]:
# 고유한 유저, 타이틀을 찾아내는 코드
user_unique = movie_rate['user_id'].unique()
title_unique = movie_rate['title'].unique()

# 유저, 타이틀 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
title_to_idx = {v:k for k,v in enumerate(title_unique)}

In [85]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['jeena'])    # 6040명의 유저 중 마지막으로 추가된 유저이니 6039이 나와야 합니다. 
print(title_to_idx['mummy, the (1999)'])

6039
710


In [86]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = movie_rate['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(movie_rate):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    movie_rate['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# title_to_idx을 통해 title 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_title_data = movie_rate['title'].map(title_to_idx.get).dropna()
if len(temp_title_data) == len(movie_rate):
    print('title column indexing OK!!')
    movie_rate['title'] = temp_title_data
else:
    print('title column indexing Fail!!')

movie_rate

user_id column indexing OK!!
title column indexing OK!!


,user_id,count,title
0,0,5,0
1,1,5,0
2,2,4,0
3,3,4,0
4,4,5,0
...,...,...,...
0,6039,5,27
1,6039,5,10
2,6039,5,385
3,6039,5,33


## 4) CSR matrix를 직접 만들어 봅시다.

In [88]:
from scipy.sparse import csr_matrix

num_user = movie_rate['user_id'].nunique()
num_title = movie_rate['title'].nunique()

csr_data = csr_matrix((movie_rate['count'], (movie_rate.user_id, movie_rate.title)), shape= (num_user, num_title))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.


In [89]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [117]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=150, regularization=0.01, use_gpu=False, iterations=40, dtype=np.float32)

In [118]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [119]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/40 [00:00<?, ?it/s]

## 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.


* 모델 학습이 끝났습니다! 이제 2가지를 아래 사항을 살펴보도록 하겠습니다.

1. jeena 벡터와 'mummy, the (1999)'의 벡터를 어떻게 만들고 있는지
2. 두 벡터를 곱하면 어떤 값이 나오는지

In [120]:
jeena, mummy = user_to_idx['jeena'], title_to_idx['mummy, the (1999)']
jeena_vector, mummy_vector = als_model.user_factors[jeena], als_model.item_factors[mummy]

print('슝=3')

슝=3


In [121]:
jeena_vector

array([ 1.31821781e-01,  1.86973721e-01, -3.38929415e-01,  5.28776683e-02,
        2.61020362e-01, -2.72277817e-02,  7.51435757e-01,  3.92047456e-03,
        2.81949729e-01,  5.21100402e-01, -2.63867050e-01,  1.99006796e-01,
       -4.51106913e-02,  5.64985693e-01, -2.22926140e-01, -1.64581671e-01,
        4.72614855e-01, -2.93255419e-01,  2.14954861e-03, -4.33138967e-01,
       -3.12646002e-01, -8.15088823e-02, -3.96458544e-02, -4.30929780e-01,
        2.22267076e-01, -2.31198505e-01,  7.40576386e-01, -1.42270461e-01,
        1.88991830e-01, -4.70389336e-01,  5.79773545e-01, -2.23485708e-01,
       -1.92696363e-01,  1.08580619e-01, -3.07265550e-01,  4.31263857e-02,
        5.69015294e-02,  5.18869340e-01, -1.10182956e-01, -1.75417393e-01,
        1.36602625e-01,  8.46464559e-02, -4.46182489e-01,  2.67624795e-01,
        1.33007005e-01,  7.58612975e-02, -1.04525588e-01, -7.22425461e-01,
        3.79512519e-01, -6.61532432e-02,  1.90603539e-01,  2.41624102e-01,
       -1.09434925e-01, -

In [122]:
mummy_vector

array([ 0.01419979,  0.00293113,  0.00077233,  0.00666677,  0.0333412 ,
        0.02321473,  0.0341106 , -0.03276315, -0.00731801,  0.02261259,
       -0.00928393,  0.01181663, -0.00482073,  0.0239321 , -0.02115093,
        0.00115244, -0.00138578, -0.00549922,  0.01179563, -0.00242168,
       -0.02319155, -0.00841193, -0.00519511, -0.01926384,  0.01631275,
        0.00819412,  0.02092624,  0.00166316,  0.01759383, -0.00576792,
        0.00943319,  0.00503727, -0.02011598,  0.01540332, -0.00220102,
       -0.01750485,  0.03930606, -0.01906009, -0.00374131, -0.00410142,
       -0.01690103, -0.00458627, -0.0106666 , -0.00407586,  0.02416428,
        0.01100213,  0.0215044 , -0.03044956,  0.02607832,  0.00036524,
       -0.01005124, -0.00971046, -0.00150723,  0.01681699,  0.02965365,
       -0.00511439,  0.02869474, -0.00380024,  0.01586285, -0.01048401,
       -0.02616735,  0.00098355,  0.01523311,  0.00389759, -0.00214069,
        0.03050959,  0.01460569, -0.00502376, -0.00057949, -0.03

###  내가 선호하는 5가지 영화 중 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악하기

In [123]:
# jeena과 mummy를 내적하는 코드
np.dot(jeena_vector, mummy_vector)

0.43357405

### 그 외의 영화를 골라 훈련된 모델이 예측한 나의 선호도를 파악하기

In [124]:
sixsense = title_to_idx['sixth sense, the (1999)']
sixsense_vector = als_model.item_factors[sixsense]
np.dot(jeena_vector, sixsense_vector)

0.008490151

In [129]:
mulan = title_to_idx['mulan (1998)']
mulan_vector = als_model.item_factors[mulan]
np.dot(jeena_vector, mulan_vector)

0.27989957

## 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

#### mummy, the (1999)

In [125]:
#title_to_idx 를 뒤집어, index로부터 title를얻는 dict를 생성합니다. 
idx_to_title = {v:k for k,v in title_to_idx.items()}

def get_similar_movie(title_name: str):
    title_id = title_to_idx[title_name]
    similar_title = als_model.similar_items(title_id)
    similar_title = [idx_to_title[i[0]] for i in similar_title]
    return similar_title

get_similar_movie('mummy, the (1999)')

['mummy, the (1999)',
 'blade (1998)',
 'rush hour (1998)',
 'con air (1997)',
 'house on haunted hill, the (1999)',
 'mask of zorro, the (1998)',
 'goldeneye (1995)',
 '13th warrior, the (1999)',
 "bram stoker's dracula (1992)",
 'rock, the (1996)']

In [155]:
# 장르확인
similar_mummy=['mummy, the (1999)',
 'blade (1998)',
 'rush hour (1998)',
 'con air (1997)',
 'house on haunted hill, the (1999)',
 'mask of zorro, the (1998)',
 'goldeneye (1995)',
 '13th warrior, the (1999)',
 "bram stoker's dracula (1992)",
 'rock, the (1996)']

movies.loc[movies['title'].isin(similar_mummy)]

,movie_id,title,genre
9,10,goldeneye (1995),Action|Adventure|Thriller
724,733,"rock, the (1996)",Action|Adventure|Thriller
1318,1339,bram stoker's dracula (1992),Horror|Romance
1513,1552,con air (1997),Action|Adventure|Thriller
1937,2006,"mask of zorro, the (1998)",Action|Adventure|Romance
2098,2167,blade (1998),Action|Adventure|Horror
2204,2273,rush hour (1998),Action|Thriller
2548,2617,"mummy, the (1999)",Action|Adventure|Horror|Thriller
2757,2826,"13th warrior, the (1999)",Action|Horror|Thriller
2926,2995,"house on haunted hill, the (1999)",Horror


* mummy의 장르가 Action|Adventure|Horror|Thriller 이므로 추천받은 비슷한 영화들도 장르가 Action, Horror, Thriiler이다.

#### beauty and the beast (1991)

In [126]:
get_similar_movie('beauty and the beast (1991)')

['beauty and the beast (1991)',
 'lion king, the (1994)',
 'aladdin (1992)',
 'mulan (1998)',
 'hunchback of notre dame, the (1996)',
 'anastasia (1997)',
 'tarzan (1999)',
 'cinderella (1950)',
 'antz (1998)',
 'little mermaid, the (1989)']

In [156]:
similar_beauty =['beauty and the beast (1991)',
 'lion king, the (1994)',
 'aladdin (1992)',
 'mulan (1998)',
 'hunchback of notre dame, the (1996)',
 'anastasia (1997)',
 'tarzan (1999)',
 'cinderella (1950)',
 'antz (1998)',
 'little mermaid, the (1989)']

movies.loc[movies['title'].isin(similar_beauty)]

,movie_id,title,genre
360,364,"lion king, the (1994)",Animation|Children's|Musical
584,588,aladdin (1992),Animation|Children's|Comedy|Musical
591,595,beauty and the beast (1991),Animation|Children's|Musical
773,783,"hunchback of notre dame, the (1996)",Animation|Children's|Musical
1009,1022,cinderella (1950),Animation|Children's|Musical
1642,1688,anastasia (1997),Animation|Children's|Musical
1838,1907,mulan (1998),Animation|Children's
2012,2081,"little mermaid, the (1989)",Animation|Children's|Comedy|Musical|Romance
2225,2294,antz (1998),Animation|Children's
2618,2687,tarzan (1999),Animation|Children's


* beauty and the beast의 장르가 Animation|Children's|Musical 이므로 추천받은 비슷한 영화들도 장르가 다 Animation|Children's 이다.

## 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [127]:
user = user_to_idx['jeena']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(330, 0.54524356),
 (39, 0.31966445),
 (34, 0.27989957),
 (30, 0.22989272),
 (841, 0.22317718),
 (16, 0.22112866),
 (191, 0.22005439),
 (384, 0.21421024),
 (316, 0.19948311),
 (329, 0.19459166),
 (35, 0.19289948),
 (453, 0.19208768),
 (248, 0.1822484),
 (997, 0.17945264),
 (40, 0.178812),
 (641, 0.17312007),
 (8, 0.17310603),
 (120, 0.17245841),
 (25, 0.17096971),
 (116, 0.16477314)]

In [128]:
[idx_to_title[i[0]] for i in movie_recommended]

['lion king, the (1994)',
 'apollo 13 (1995)',
 'mulan (1998)',
 'antz (1998)',
 'prince of egypt, the (1998)',
 'tarzan (1999)',
 'little mermaid, the (1989)',
 'jerry maguire (1996)',
 'misery (1990)',
 'anastasia (1997)',
 'hunchback of notre dame, the (1996)',
 'boogie nights (1997)',
 'good will hunting (1997)',
 'scream (1996)',
 'toy story (1995)',
 "mr. holland's opus (1995)",
 'snow white and the seven dwarfs (1937)',
 'raiders of the lost ark (1981)',
 'pocahontas (1995)',
 'dances with wolves (1990)']

### 내가 선호하는 영화 5개의 장르 확인

In [154]:
movies.loc[movies['title'].isin(my_favorite)]

,movie_id,title,genre
584,588,aladdin (1992),Animation|Children's|Comedy|Musical
591,595,beauty and the beast (1991),Animation|Children's|Musical
1636,1682,"truman show, the (1998)",Drama
1672,1721,titanic (1997),Drama|Romance
2548,2617,"mummy, the (1999)",Action|Adventure|Horror|Thriller


### 추천받은 내가 가장 좋아할 만한 영화들의 장르 확인

In [153]:
v=[idx_to_title[i[0]] for i in movie_recommended]
movies.loc[movies['title'].isin(v)]

,movie_id,title,genre
0,1,toy story (1995),Animation|Children's|Comedy
47,48,pocahontas (1995),Animation|Children's|Musical|Romance
61,62,mr. holland's opus (1995),Drama
148,150,apollo 13 (1995),Drama
360,364,"lion king, the (1994)",Animation|Children's|Musical
586,590,dances with wolves (1990),Adventure|Drama|Western
590,594,snow white and the seven dwarfs (1937),Animation|Children's|Musical
773,783,"hunchback of notre dame, the (1996)",Animation|Children's|Musical
1180,1198,raiders of the lost ark (1981),Action|Adventure
1372,1393,jerry maguire (1996),Drama|Romance


# 회고

### **내가 선호하는 5가지 영화의 장르**
|영화 제목|장르|
|:---:|:---|
|truman show, the (1998)|Drama|
|titanic (1997)|Drama/Romance|
|beauty and the beast (1991)|Animation/Children's/Musical|
|aladdin (1992)|Animation/Children's/Comedy/Musical|
|mummy, the (1999)|Action/Adventure/Horror/Thriller|

:  애니메이션 2개, 로맨스드라마 1개, 드라마1개, 호러 1개

==> 이렇게 장르가 분포되있어서 그런지 추천받은 영화들이 대부분 애니메이션이다. 그다음으로 드라마가 많고 호러영화가 몇개 존재한다.

### **내가 선호하는 5가지 영화 중 하나와 그 외의 영화를 골라 훈련된 모델이 예측한 나의 선호도를 파악**<br>
내가 선호하는 영화 : mummy, the (1999) , 그 외의 영화 : sixth sense, the (1999)



* AlternatingLeastSquares 클래스의 init 파라미터 
1. factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지 
2. regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지 
3. use_gpu : GPU를 사용할 것인지 
4. iterations : epochs와 같은 의미입니다. 데이터를 몇 번 반복해서 학습할 것인지 <br>
=> 1,4를 늘릴수록 학습데이터를 잘 학습하게 되지만 과적합의 우려가 있으니 좋은 값을 찾아야된다고 한다.

* 파라미터 조정
1. als_model에서 `factors=100, regularization=0.01, use_gpu=False, iterations=15`로 하니 jeena과 mummy를 내적한 결과가 0.31482702이 나왔다. sixthsense와 내적한 결과는 0.009가 나왔다.
2. `factors=150, regularization=0.01, use_gpu=False, iterations=40` => jeena과 mummy를 내적한 결과가 0.43357405이 나왔다. sixthsense와 내적한 결과는 0.008가 나왔다.

    * 파라미터 factors, iterations를 늘리니 내가 선호하는 영화 중 하나에 대해 모델이 예측한 나의 선호도는 증가하고 그 외의 영화에 대한 모델이 예측한 나의 선호도는 줄어들었다.
    * 선호하는 영화 5개 중에 애니메이션이 2개가 존재해서그런지 sixth sense(0.008)에 대한 선호도보단 애니메이션인 mulan (1998)의 선호도(0.27989957)가 더 높게 나왔다.